In [10]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Load the pre-trained BERT model and tokenizer
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'transformers.models.bert.modeling_tf_bert'

In [11]:
import os
os.environ["USE_TF"] = "1"

In [6]:
# Function to tokenize input sentences
def encode_sentences(sentences, tokenizer, max_length=128):
    
    inputs = tokenizer(
        sentences,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        return_attention_mask=True,
    )
    
    # Ensure that input_ids and attention_mask are returned properly
    input_ids = inputs.get('input_ids')
    attention_mask = inputs.get('attention_mask')

    return input_ids, attention_mask

sentences = ["I love programming", "The weather is nice today", "I have an exam tomorrow"]
labels = ["positive", "neutral", "negative"]

# Encode labels (3 categories)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [57]:

# Define the model architecture
input_ids_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT model
bert_output = bert_model(input_ids_layer, attention_mask=attention_mask_layer)
pooled_output = bert_output.pooler_output

# Add a classification layer
output = tf.keras.layers.Dense(1, activation='sigmoid')(pooled_output)

# Create the Keras model
model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Print the input shapes for debugging
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Mask shape: {attention_mask.shape}")

Input IDs shape: (2, 128)
Attention Mask shape: (2, 128)


In [58]:
# Dummy labels (1 for positive, 0 for negative) for binary classification
labels = [1, 0]
labels = tf.convert_to_tensor(labels)

# Train the model
model.fit([input_ids, attention_mask], labels, epochs=3, batch_size=8)

Epoch 1/3
1/1 [==============================] - 10s 10s/step - loss: 0.8081 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 1s 752ms/step - loss: 0.6488 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 1s 672ms/step - loss: 0.4834 - accuracy: 0.5000


In [54]:
test_sentences = ["This is a test sentence."]
test_input_ids, test_attention_mask = encode_sentences(test_sentences, tokenizer)

predictions = model.predict([test_input_ids, test_attention_mask])
print(predictions)

1/1 [==============================] - 1s 1s/step
[[0.34069943]]
